In [ ]:
import numpy as np
import pandas as pd 
import time
import requests
from fake_useragent import UserAgent

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import lxml
from lxml.html import fromstring


ua = UserAgent()
url = "주소"
headers = {'User-Agent' : ua.random}

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")

# 파라미터

In [ ]:
KEYWORDS = ['우유', '참외', '상추', '삼겹살']

# URL 크롤링 (by lxml)

In [ ]:
def get_info_about_url(base_url):
    req = requests.get(base_url)
    doc = fromstring(req.text)
    productnames = [element.text_content() for element in doc.xpath('//*[@id="__next"]/div/div[2]/div/div[3]/div[1]/ul/div/div/li/div/div[2]/div[1]/a')]
    dates = [element.text_content().split(' ')[1] for element in doc.xpath('//*[@id="__next"]/div/div[2]/div/div[3]/div[1]/ul/div/div/li/div/div[2]/div[5]/span[1]')]
    review_nums = [element.text_content() for element in doc.xpath('//*[@id="__next"]/div/div[2]/div/div[3]/div[1]/ul/div/div/li/div/div[2]/div[5]/a/em')]
    urls = [element.get('href') for element in doc.xpath('//*[@id="__next"]/div/div[2]/div/div[3]/div[1]/ul/div/div/li/div/div[2]/div[1]/a')]

            
    if len(review_nums) < 5:
        while len(review_nums) != len(productnames):
            review_nums.append('0')
            
    return productnames, dates, review_nums, urls

In [ ]:
df_url = pd.DataFrame()
t1 = time.time()

for idx, KEYWORD in enumerate(KEYWORDS):
    PAGE = 1

    while True:
        base_url = f'https://search.shopping.naver.com/search/all?exagency=true&frm=NVSHMDL&origQuery={KEYWORD}&pagingIndex={PAGE}&pagingSize=5&productSet=model&query={KEYWORD}&sort=review&timestamp=&viewType=list'

        # 정보수집
        productnames, dates, review_nums, urls = get_info_about_url(base_url)
        search_words = [KEYWORD] * len(productnames)

        # Stopping condition
        if len(productnames) == 0:
            try:
                print(f'== 키워드: {KEYWORD}의 {PAGE}페이지에 정보가 존재하지 않습니다. url 오류일 수 있으니 10초 후 재접근합니다.')
                time.sleep(10)
                productnames, dates, review_nums, urls = get_info_about_url(base_url)
                search_words = [KEYWORD] * len(productnames)

            except:
                print(f'== 키워드: {KEYWORD}의 {PAGE}페이지부터 정보가 존재하지 않는 것으로 판단됩니다. 다음 키워드로 넘어갑니다. == 경과시간: {round(time.time()-t1)}초')
                break

        # Append df
        tmp = pd.DataFrame({'상품명':productnames,
                           '등록일':dates,
                            '리뷰개수':review_nums,
                           'url':urls,
                           '검색키워드':search_words})
        df_url = df_url.append(tmp, ignore_index=True)
        time.sleep(0.5)

        if '0' in review_nums:
            print(f'{PAGE}페이지에 리뷰가 0개인 항목이 존재합니다. 이하 항목은 모두 리뷰가 0개로 판단되어 조기종료합니다.')
            break

        # log
        if PAGE % 10 == 0:
            print(f'== 키워드: {KEYWORD} == 페이지: {PAGE} 완료 == 경과시간: {round(time.time()-t1)}초')

        # add PAGE
        PAGE += 1

    # log2
    KEYWORD_NAME = KEYWORD.replace('/','')
    df_url.to_csv(f'네이버쇼핑몰_url_{KEYWORD_NAME}까지.csv', index=False)
    print(f'== 키워드: {KEYWORD} 완료({idx+1}/{len(KEYWORDS)})!! == 경과시간: {round(time.time()-t1)}초 == 중간저장완료\n')
    time.sleep(5)
    
df_url = df_url[df_url['리뷰개수'] != '0'].reset_index(drop=True)

# 세부정보수집 (selenium)

In [ ]:
def get_info_about_detail():

    contents = [element.text for element in driver.find_elements_by_xpath('//*[@id="section_review"]/ul/li/div[2]/div[1]/p')]
    productnames = [df_url.loc[url_idx,'상품명']] * len(contents)
    dates = [element.text for element in driver.find_elements_by_xpath('//*[@id="section_review"]/ul/li/div[1]/span[4]')]
    writers = [element.text for element in driver.find_elements_by_xpath('//*[@id="section_review"]/ul/li/div[1]/span[3]')]
    scores = [score] * len(contents)
    headlines = [element.text for element in driver.find_elements_by_xpath('//*[@id="section_review"]/ul/li/div[2]/div/em')]
    urls = [url] * len(contents)
    shoppingmalls = [element.text for element in driver.find_elements_by_xpath('//*[@id="section_review"]/ul/li/div[1]/span[2]')]
    search_words = [df_url.loc[url_idx, '검색키워드']] * len(contents)

    tmp = pd.DataFrame({'작성자':writers,
                           '작성일자':dates,
                           '상품명':productnames,
                           '평점':scores,
                           '구매쇼핑몰':shoppingmalls,
                           '제목':headlines,
                           '본문':contents,
                           'url':urls,
                           '검색키워드':search_words})
    return tmp



def move_page(): # 다음페이지로 이동할 수 있도록 화면 이동
    location = driver.find_element_by_xpath('//*[@id="section_review"]/div[3]/a[1]')
    driver.execute_script(f"window.scrollTo(0, {location.location['y']-300})") 
    time.sleep(0.5)
    driver.find_element_by_xpath(f'//*[@id="section_review"]/div[3]/a[{page}]').click()
    time.sleep(0.5)

In [ ]:
#df_url = pd.read_csv('../데이터/네이버쇼핑몰리뷰크롤링/네이버쇼핑몰_url_data_sample.csv')
df_url

In [ ]:
df_final = pd.DataFrame()
t1 = time.time()
error_idx = []
driver = webdriver.Chrome('chromedriver_91.exe', options=options)
score_1_path = '//*[@id="section_review"]/div[2]/div[2]/ul/li[6]/a' # 1점
score_2_path = '//*[@id="section_review"]/div[2]/div[2]/ul/li[5]/a' # 2점
score_3_path = '//*[@id="section_review"]/div[2]/div[2]/ul/li[4]/a' # 3점
score_4_path = '//*[@id="section_review"]/div[2]/div[2]/ul/li[3]/a' # 4점
score_5_path = '//*[@id="section_review"]/div[2]/div[2]/ul/li[2]/a' # 5점

for url_idx in range(df_url.shape[0]):
    
    try:
        # url 접속
        url = df_url.loc[url_idx, 'url']
        driver.get(url)    
        print(f'\n{url_idx+1}번째 url 시작 == 경과시간:{round(time.time()-t1)}초')

        # 쇼핑몰 리뷰 있는 페이지까지 이동
        location = driver.find_element_by_xpath('//*[@id="snb"]/ul/li[2]/a')
        driver.execute_script(f"window.scrollTo(0, {location.location['y']-200})") 
        time.sleep(1)

        # 쇼핑몰리뷰 클릭
        driver.find_element_by_xpath('//*[@id="snb"]/ul/li[2]/a').click()
        time.sleep(0.5)

        # 1점, 2점, 3점 위치로 이동
        location = driver.find_element_by_xpath(score_1_path)
        driver.execute_script(f"window.scrollTo(0, {location.location['y']-200})") 

        # 리뷰 크롤링
        for score, score_element in zip([1,2,3], [score_1_path, score_2_path, score_3_path]):

            try: # 1점, 2점, 3점 리뷰가 없을 수도 있다.
                # 각 해당하는 점수를 클릭
                location = driver.find_element_by_xpath(score_1_path)
                driver.execute_script(f"window.scrollTo(0, {location.location['y']-200})") 
                time.sleep(0.5)
                driver.find_element_by_xpath(score_element).click()
                time.sleep(0.5)

                loop = True
                page_count = 1

                while (loop):

                    # Case1. "다음"이 존재하는 것
                    try: # 1page 단위에서 "다음"이 존재하는 것
                        if driver.find_element_by_xpath(f'//*[@id="section_review"]/div[3]/a[11]').text == '다음':
                            for page in range(2, 12):
                                tmp = get_info_about_detail()
                                df_final = df_final.append(tmp, ignore_index=True)
                                move_page()
                                page_count += 1

                        elif driver.find_element_by_xpath(f'//*[@id="section_review"]/div[3]/a[12]').text == '다음': # 11~20, 21~30처럼 다음이 존재하는 경우
                            for page in range(3, 13):
                                tmp = get_info_about_detail()
                                df_final = df_final.append(tmp, ignore_index=True)
                                
                                if driver.find_element_by_xpath(f'//*[@id="section_review"]/div[3]/a[{page}]').text == '100':
                                    print(f'리뷰개수가 100페이지에 도달하면 더이상 네이버에서 제공하지 않습니다.')
                                    loop = False
                                    break
                                
                                move_page()
                                page_count += 1

                    # Case2. "다음"이 존재하지 않는 경우
                    except: 

                        try:
                            if driver.find_element_by_xpath(f'//*[@id="section_review"]/div[3]/a[1]').text == '이전': # 2-1. "이전"이 존재하는 경우 71~73같은 형태

                                for page in range(3, 13):
                                    try:
                                        tmp = get_info_about_detail()
                                        df_final = df_final.append(tmp, ignore_index=True)
                                        move_page()
                                        page_count += 1

                                    except:
                                        loop = False
                                        break


                            else: # 2-1. "이전"이 존재하지 않는 경우 1~8같은 형태
                                for page in range(2, 12):
                                    try:
                                        tmp = get_info_about_detail()
                                        df_final = df_final.append(tmp, ignore_index=True)
                                        move_page()
                                        page_count += 1

                                    except:
                                        loop = False
                                        break

                        except: # Case3. 1페이지만 있는 경우 상태바가 아예 없음
                            tmp = get_info_about_detail()
                            df_final = df_final.append(tmp, ignore_index=True)
                            loop = False


                print(f'{score}점 수집완료!')

            except:
                print(f'{score}점 리뷰가 존재하지 않습니다!')


        if (url_idx+1) % 100 == 0:
            df_final.to_csv(f'네이버쇼핑몰_세부정보_{url_idx+1}개까지.csv', index=False)
    except:
        print(f'== {url_idx+1}번째 url에 오류 발견 == 수집하지 않습니다. ==')
        error_idx.append(url_idx)

In [ ]:
df_final.to_excel('../데이터/코티티_네이버쇼핑몰_크롤링_샘플.xlsx', index=False)